In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/hivep9
!hdfs dfs -ls /tmp/
!hdfs dfs -copyFromLocal tbl0.csv /tmp/hivep9/
!hdfs dfs -copyFromLocal tbl1.csv /tmp/hivep9/

Found 3 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 03:16 /tmp/hivep8
Found 4 items
drwxrwx---   - root supergroup          0 2020-02-11 03:08 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-11 03:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-11 03:16 /tmp/hivep8
drwxr-xr-x   - root supergroup          0 2020-02-11 03:20 /tmp/hivep9


In [3]:
%%hive
DROP TABLE IF EXISTS tbl0;
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;

DROP TABLE IF EXISTS tbl0;
OK
Time taken: 19.675 seconds
CREATE TABLE tbl0 (
    c1 INT,
    c2 STRING,
    c3 INT,
    c4 DATE,
    c5 ARRAY<CHAR(1)>, 
    c6 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 2.109 seconds
LOAD DATA LOCAL INPATH 'tbl0.csv' INTO TABLE tbl0;
Loading data to table default.tbl0
OK
Time taken: 2.917 seconds


In [5]:
%%hive
DROP TABLE IF EXISTS tbl1;
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;

DROP TABLE IF EXISTS tbl1;
OK
Time taken: 0.03 seconds
CREATE TABLE tbl1 (
    c1 INT,
    c2 INT,
    c3 STRING,
    c4 MAP<STRING, INT>
)
ROW FORMAT DELIMITED 
FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
MAP KEYS TERMINATED BY '#'
LINES TERMINATED BY '\n';
OK
Time taken: 0.25 seconds
LOAD DATA LOCAL INPATH 'tbl1.csv' INTO TABLE tbl1;
Loading data to table default.tbl1
OK
Time taken: 1.908 seconds


In [11]:
%%hive
DROP TABLE IF EXISTS t4;
CREATE TABLE t4 AS
WITH t0 AS(
SELECT 
 c1,letra,numero
FROM tbl1 LATERAL VIEW explode(c4) adTable AS letra,numero
)
SELECT 
 tbl0.c1,tbl0.c2,t0.numero
FROM tbl0 
LEFT JOIN t0 ON (tbl0.c1= t0.c1 and tbl0.c2 =t0.letra);

INSERT OVERWRITE DIRECTORY '/tmp/hivep9'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
COLLECTION ITEMS TERMINATED BY ':'
SELECT 
    *
FROM t4;

DROP TABLE IF EXISTS t4;
OK
Time taken: 0.996 seconds
CREATE TABLE t4 AS
WITH t0 AS(
SELECT 
 c1,letra,numero
FROM tbl1 LATERAL VIEW explode(c4) adTable AS letra,numero
)
SELECT 
 tbl0.c1,tbl0.c2,t0.numero
FROM tbl0 
LEFT JOIN t0 ON (tbl0.c1= t0.c1 and tbl0.c2 =t0.letra);
Query ID = root_20200211035315_80fdfa7f-6aec-4b4c-9323-047a9e7eab87
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
2020-02-11 03:53:31	Starting to launch local task to process map join;	maximum memory = 477626368
2020-02-11 03:53:35	Dump the side-table for tag: 1 with group count:

In [12]:
!hdfs dfs -cat /tmp/hivep9/000000_0 

1,D,3
2,C,0
3,D,1
4,D,5
5,C,2
6,A,4
7,B,5
8,C,5
9,B,4
10,B,3


In [18]:
!hdfs dfs -copyToLocal /tmp/hivep9/000000_0 output;

In [8]:
%%hive
DROP TABLE t4;
DROP TABLE t0;

DROP TABLE tbl0;
FAILED: ParseException line 2:0 missing EOF at 'DROP' near 'pt8'
DROP TABLE pt8;
OK
Time taken: 0.209 seconds


In [19]:
!rm *.csv *.log

rm: cannot remove 'derby.log': Text file busy


In [21]:
%hive_quit

AttributeError: 'NoneType' object has no attribute 'close'